<a href="https://colab.research.google.com/github/nanekeshishyan/iLur-News-Classification/blob/main/iLurNewsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
import string

train_data = pd.read_csv('/kaggle/input/ilur-news-classification-2023/train.csv')
test_data = pd.read_csv('/kaggle/input/ilur-news-classification-2023/test_set.csv')

stop_words = [  'այդ',    'այլ',    'այն',    'այս',    'դու',    'դուք',    'եմ',    'են',    'ենք',    'ես',    'է',    'էի',    'էին',    'էինք',    'էիր',    'էիք',    'էր',    'ըստ',    'թե',    'ի',    'ին',    'իսկ',    'իր',    'կամ',    'համար',    'հետ',    'հետո',    'մենք',    'մեջ',    'մի',    'ն',    'նա',    'նաև',    'նրա',    'նրանք',    'որ',    'որը',    'որոնք',    'որպես',    'ու',    'ում',    'պիտի',    'վրա',    'և' , 'նույնը', 'այստեղ', 'հիմա', 'ավելի լավ', 'կրկին', 'ինչ', 'լավ', 'նա', 'չի կարող', 'դա', 'հիմա', 'այստեղ', 'դեռ', 'կարող է', 'կար', 'ոչ', 'որովհետև', 'և', 'դու', 'կամ', 'կար', 'եթե', 'նրան', 'առաջ', 'արդեն', 'քան', 'կարող է', 'վրա', 'րան', 'բոլորը', 'քանի որ', 'ինչ', 'ինչ-որ բան', 'սա', 'նույնիսկ', 'մենք', 'այն', 'նա', 'ըստ', 'մեզ', 'բացարձակապես', 'կային', 'այն', 'նրա', 'իրեն', ' երբեմն', 'հետ', 'ես', 'դա', 'երկու', 'սրանք', 'ով', 'մեջ', 'համար' , 'միջևում', 'դու', 'կարծես', 'իմը', 'այդպես', 'տակ' ,'պետք է', 'միայն', 'երբեք', 'բայց', 'դեպի', 'ինչի համար', 'համար', 'մինչև', 'վերջապես', 'վերջում' , 'այստեղ', 'առանց', 'որտեղ', 'նրանք', 'երեք', 'նրանից', 'ոչինչ', 'ինքս', 'լավ', 'ավելի', 'շատ', 'գրեթե', 'լինիր', 'ուրիշ', 'այնտեղ', 'մոտ', 'սակայն', 'այո', 'այդպիսին', 'իհարկե', 'առնվազն', 'ավելի շատ', 'նույնպես', 'հանկարծ', 'երբ', 'կա', 'հետո', 'միայն', 'մի քիչ', 'միջոցով', 'ինչու', 'մեկ անգամ', 'իմ', 'արդյո՞ք', 'մեկ', 'հետ', 'ներս', 'միշտ']
def preprocess_text(text):
    # удаление знаков препинания
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    # исключение стоп-слов
    text = ' '.join(filter(lambda w: w not in stop_words, text.split()))
    return text


train_data['text'] = train_data['text'].apply(preprocess_text)
test_data['text'] = test_data['text'].apply(preprocess_text)


X_train, X_val, y_train, y_val = train_test_split(train_data['text'], train_data['category'], test_size=0.2, random_state=42, stratify=train_data['category'])


vectorizer = TfidfVectorizer(min_df=5, max_df=0.5, ngram_range=(1, 2), sublinear_tf=True)
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)
X_test = vectorizer.transform(test_data['text'])



classificator = LinearSVC(penalty = 'l2', C = 1)
classificator.fit(X_train, y_train)



y_train_pred = classificator.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("train Accuracy:", train_accuracy)
y_val_pred = classificator.predict(X_val)
y_test_pred = classificator.predict(X_test)



accuracy = accuracy_score(y_val, y_val_pred)
print("Accuracy:", accuracy)
report = classification_report(y_val, y_val_pred)
print(report)



dataFrame = pd.DataFrame({'id': test_data['id'], 'category': y_test_pred})
dataFrame.to_csv('/kaggle/working/submission.csv', index=False)